In [1]:
import os
#import pandas as pd
import modin.pandas as mpd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split,  cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.externals import joblib

#from keras.models import Sequential
#from keras.layers import Dense, Activation, Dropout
#from keras.wrappers.scikit_learn import KerasClassifier # keras wrapper for sklearn



Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:64310 to respond...
Waiting for redis server at 127.0.0.1:41139 to respond...
Starting the Plasma object store with 6.00 GB memory.


In [13]:
%%timeit
data_s = pd.read_csv('train_sample.csv')
#.fillna('')

150 ms ± 8.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
data_s.sort_values(by='attributed_time', ascending = False).head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
28970,346097,19,16,0,213,2017-11-09 15:27:07,2017-11-09 15:28:15,1
5111,303875,71,1,19,3,2017-11-09 15:13:48,2017-11-09 15:17:13,1
93842,346361,115,1,22,203,2017-11-09 15:14:10,2017-11-09 15:15:34,1
39247,305220,19,581,24,213,2017-11-09 15:03:23,2017-11-09 15:05:43,1
27035,75443,9,1,19,466,2017-11-09 14:12:55,2017-11-09 14:53:22,1


In [9]:
import dask.dataframe as dd
data_s = dd.read_csv('train_sample.csv')
results=data_s.groupby('user').apply(calc_HR_ind_dsk).compute()

KeyError: 'user'

In [4]:
data_s

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
npartitions=1,,,,,,,,
,int64,int64,int64,int64,int64,object,object,int64
,...,...,...,...,...,...,...,...


In [22]:
data_s.sort_values(by='attributed_time', ascending = False).head()

AttributeError: 'DataFrame' object has no attribute 'sort_values'

In [21]:
# Checking for NANs
for col in data_s.columns:
    print('{} contains {:,} NANs'.format(col,len(data_s[data_s[col].isnull()])))

ip contains 0 NANs
app contains 0 NANs
device contains 0 NANs
os contains 0 NANs
channel contains 0 NANs
click_time contains 0 NANs
attributed_time contains 99,773 NANs
is_attributed contains 0 NANs


Only attributed time has NANs, but I am not using this column

In [7]:
# Removing attributed_time
data_s = data_s.drop('attributed_time', axis = 1)

In [12]:
data_s.head()

,ip,app,device,os,channel,click_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,0
1,105560,25,1,17,259,2017-11-07 13:40:27,0
2,101424,12,1,19,212,2017-11-07 18:05:24,0
3,94584,13,1,13,477,2017-11-07 04:58:08,0
4,68413,12,1,1,178,2017-11-09 09:00:09,0


In [14]:
# Convert click_time to datetime format
data_s.click_time = dd.to_datetime(data_s.click_time, errors = 'ignore')

In [16]:
# Determine and coding time of the day the site was accessed
# 1 = Night, 2 = Morning, 3 = Afternoon, 4 = Evening
data_s = data_s.assign(session = pd.cut(data_s.click_time.dt.hour,[-1,6,12,18,24],labels=[1, 2, 3,4 ]))
#data_s.session = data_s.session.astype(np.int64)
#data_s = data_s.drop('click_time', 1)

TypeError: Column assignment doesn't support type Categorical

In [138]:
data_s.head()

,ip,app,device,os,channel,is_attributed,session
0,87540,12,1,13,497,0,2
1,105560,25,1,17,259,0,3
2,101424,12,1,19,212,0,3
3,94584,13,1,13,477,0,1
4,68413,12,1,1,178,0,2


In [139]:
data_s = data_s[['ip', 'app', 'device', 'os', 'channel', 'session','is_attributed']]

In [ ]:
from collections import defaultdict
d = defaultdict(LabelEncoder)
# Encoding the variable
fit = test.apply(lambda x: d[x.name].fit_transform(x))

# Inverse the encoded
#fit.apply(lambda x: d[x.name].inverse_transform(x))
# Using the dictionary to label future data
data_s.apply(lambda x: d[x.name].transform(x))

In [ ]:
data_s.head()

In [ ]:
data_s = data_s.reset_index(drop = False)
data_s = data_s.rename(columns={'index':'click_id'})

In [ ]:
data_s.is_attributed.unique()

In [ ]:
data_s = data_s.astype(np.float64)

In [ ]:
data_s.sort_values(by='ip').head()

# Aritificial Neural Network 

### Split data into target and features

In [ ]:
target = data_s.is_attributed.values
features = data_s.drop('is_attributed', 1).values
print ('Data original %d, target: %d, features: %d' % (data_s.shape[0], target.shape[0], features.shape[0]))

### Split features into training, validation, and testing sets

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(features, target, test_size= 0.25, random_state=1)
X_train, X_val, y_train, y_val    = train_test_split(X_train, y_train, test_size= 0.20, random_state=1)

X_train_names = X_train[:,0]
X_train = X_train[:,1:7]

X_test_names = X_test[:,0]
X_test = X_test[:,1:7]
                  
X_val_names = X_val[:,0]
X_val = X_val[:,1:7]

### Standarizing the features

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val   = sc.transform(X_val)
X_test  = sc.transform(X_test)

In [ ]:
X_train

### Building the initial model 

In [ ]:
def classifier_model(activator, optimizer, initializer):
    classifier = Sequential()
    classifier.add(Dense(units = 10, kernel_initializer = initializer, activation = activator, input_dim = 6))
    classifier.add(Dense(units = 10, kernel_initializer = initializer, activation = activator))
    classifier.add(Dense(units = 1,  kernel_initializer = initializer, activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier

### Fitting the model with the training set 

In [119]:
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
classifier = classifier_model(activator, optimizer, initializer)
classifier.fit(X_train, y_train, batch_size = 10, epochs = 10)

60000/60000 [==============================] - 13s - loss: 0.0126 - acc: 0.9978    
Epoch 5/10
60000/60000 [==============================] - 14s - loss: 0.0126 - acc: 0.9978    
Epoch 6/10
60000/60000 [==============================] - 11s - loss: 0.0125 - acc: 0.9978    
Epoch 7/10
60000/60000 [==============================] - 11s - loss: 0.0124 - acc: 0.9978    
Epoch 8/10
60000/60000 [==============================] - 11s - loss: 0.0124 - acc: 0.9978    
Epoch 9/10
60000/60000 [==============================] - 11s - loss: 0.0123 - acc: 0.9978    
Epoch 10/10
60000/60000 [==============================] - 13s - loss: 0.0123 - acc: 0.9978    


### Crossvalidation on training set

In [ ]:
# Using relu - as raw model above - as activator
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
cv = 10 
classifier_cv = KerasClassifier(build_fn = classifier_model, 
                                activator = activator, 
                                optimizer = optimizer,
                                initializer = initializer,
                                batch_size = 10, 
                                epochs = 10, 
                                verbose = 0)
accuracies = cross_val_score(estimator = classifier_cv, X = X_train, y = y_train, cv = cv)

mean = accuracies.mean()
variance = accuracies.std()

In [ ]:
# Mean cv accuracy 
print('Mean cv accuracy (relu) = {:.4f}% +/- {:.4f}'.format(mean *100, variance *100) )

### Validation 

In [ ]:
loss, accuracy = classifier.evaluate(X_val, y_val, batch_size = 10, verbose = 0)
print("Validation set accuracy = {:.4f}%, Loss = {:.4f}".format(accuracy* 100, loss))

In [ ]:
loss, accuracy = classifier.evaluate(X_train, y_train , batch_size = 128, verbose = 0)
print(round(accuracy*100,2))

### Optimization

Using GridSearch

In [ ]:
classifier_op = KerasClassifier(build_fn = classifier_model)

parameters = {'batch_size'  : [10, 30],
              'epochs'      : [10, 20],
              'optimizer'   : ['adam', 'rmsprop'],
              'activator'   : ['relu', 'sigmoid'],
              'initializer' : ['uniform']}

grid_search = GridSearchCV(estimator = classifier_op,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
best_accuracy

In [ ]:
best_parameters

# OPTIMIZED MODEL 

In [ ]:
activator   = 'relu'
optimizer   = 'adam'
initializer = 'uniform'
epoch = 10 
classifier_opt = classifier_model(activator, optimizer, initializer)
classifier_opt.fit(X_train, y_train, batch_size = 10, epochs = epoch, verbose = 0)

In [ ]:
# Predicting on the test set 
y_pred = classifier_opt.predict(X_test)
y_pred 

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred.round(), average='weighted')

In [ ]:
y_pred.round()

In [ ]:
y_classes = y_pred.argmax(axis=-1)
y_a = pd.DataFrame([y_classes])
y_b = pd.DataFrame([y_test])
y_c = pd.DataFrame([X_test_names])
y_a = pd.concat((y_a,y_b, y_c)).T
y_a.columns = ['predicted', 'actual','click_id']
y_a.head()

In [ ]:
# Number of mismatches within the test set
y_a['dif'] = np.where(y_a.predicted == y_a.actual, 0, 1)
y_a[y_a['dif'] == 1].actual.value_counts().sort_index()

In [ ]:
classifier_opt.save('classifier.hdf5')
joblib.dump(sc, 'scaler.pkl') 

------------------
PREDICTION ON THE TESTING DATASET

In [ ]:
from keras.models import load_model
classifier = load_model('classifier.hdf5')
from sklearn.preprocessing import StandardScaler
sc = joblib.load('scaler.pkl')

In [120]:
test = pd.read_csv('test.csv')

In [121]:
test.head()

,click_id,ip,app,device,os,channel,click_time
0,0,5744,9,1,3,107,2017-11-10 04:00:00
1,1,119901,9,1,3,466,2017-11-10 04:00:00
2,2,72287,21,1,19,128,2017-11-10 04:00:00
3,3,78477,15,1,13,111,2017-11-10 04:00:00
4,4,123080,12,1,13,328,2017-11-10 04:00:00


In [122]:
# Convert click_time to datetime format
test.click_time = pd.to_datetime(test.click_time, errors = 'ignore')

In [123]:
# Checking for NANs
for col in test.columns:
    print('{} contains {:,} NANs'.format(col,len(test[test[col].isnull()])))

click_id contains 0 NANs
ip contains 0 NANs
app contains 0 NANs
device contains 0 NANs
os contains 0 NANs
channel contains 0 NANs
click_time contains 0 NANs


In [126]:
# Determine and coding time of the day the site was accessed
# 1 = Night, 2 = Morning, 3 = Afternoon, 4 = Evening
test = test.assign(session=pd.cut(test.click_time.dt.hour,[-1,6,12,18,24],labels=[1, 2, 3,4 ]))
test = test.drop('click_time', 1)

In [127]:
test.columns

Index(['click_id', 'ip', 'app', 'device', 'os', 'channel', 'session'], dtype='object')

In [128]:
test = test[['ip', 'app', 'device', 'os', 'channel', 'session']]

In [129]:
# Using the dictionary to label future data
test.apply(lambda x: d[x.name].transform(x))

ValueError: ('y contains new labels: [     0      1      2 ..., 126409 126412 126413]', 'occurred at index ip')

In [ ]:
test = test.reset_index(drop = False)
test = test.rename(columns={'index':'click_id'})
test.head()

In [ ]:
print(len(test.channel.unique()))
test.channel.value_counts().sort_index()

In [ ]:
test.dtypes

In [ ]:
test_x_names = test.click_id
test_x = test.drop('click_id', 1)


In [ ]:
X_2016 = sc.transform(test_x)

In [ ]:
# Predicting on the test set 
predictions = classifier.predict(X_2016)
predictions

In [ ]:
pred_clas = classifier.predict_classes(X_2016)

In [ ]:
labels_ = predictions.argmax(axis=1)
#labels_ = labels_> 0.5
labels_

In [ ]:
a = pd.concat([pd.DataFrame(test_x_names), pd.DataFrame(predictions)], axis =1).rename(columns={0:'is_attributed'})        

In [ ]:
a.to_csv('First.csv', header = True, index = False)

In [ ]:
a.to_csv('primera.csv', header = True, index = False)

In [ ]:
predictions1 = predictions